In [9]:
# Load some jobs into a pandas dataframe

import pandas as  pd
pd.set_option('display.max_colwidth', 300)
pd.options.mode.chained_assignment = None


# replace with student spreadsheet

jobs = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRWmeWZs5NHptaUiotzWf8Qf9evBzdDToTvkMbn1v0ug5uqu32vPd4BwTU7DsogP9cU7xKov0ku8gYV/pub?output=csv')
jobs = jobs[['year', 'date', 'description']]
jobs.head(5)


,year,date,description
0,2025,04-03,"Elections Systems Administrator, Arapahoe County, Colorado – The Election Systems Administrator performs leadership level administrative, and professional work in carrying out a comprehensive public facing service operation. This position specifically leads and supports all areas of ballot proce..."
1,2025,03-27,"Ballot Processing Supervisor, Boulder County, Colorado– The Boulder County Clerk and Recorder’s Office is currently seeking a Ballot Processing Supervisor to join our team. This vital role supports the execution of accurate and compliant elections within Boulder County. The Ballot Processing Sup..."
2,2025,03-27,"Deputy Director, Minnesota Secretary of State–The Office of the Minnesota Secretary of State (OSS) is seeking a Deputy Elections Director that supports the Elections Director in providing division-wide strategic and administrative management, along with program and staff management, of all activ..."
3,2025,03-27,"Risk & Compliance Manager, Illinois State Board of Elections– Reporting to the Chief Information Security Officer (CISO) the Risk & Compliance Manager is responsible for: Assessing organizational risks, ensuring the agency’s alignment to information security standards, collaborating with agency ..."
4,2025,03-27,"Technology & Cybersecurity Project Manager, Boulder County, Colorado– The Boulder County Clerk and Recorders department is seeking to hire a Technology and Cybersecurity Project Manager. The Boulder County Clerk and Recorder’s Office provides Motor Vehicle, Recording, and Election services for ..."


In [ ]:
# let's try to extract information from the job description
jobs.loc[0, 'description']

'Elections Systems Administrator, Arapahoe County, Colorado – The Election Systems Administrator performs leadership level administrative, and professional work in carrying out a comprehensive public facing service operation. This position specifically leads and supports all areas of ballot processing, ballot auditing and election technology maintenance. Salary: $64,293.06 – $96,439.98 Annually. Deadline: April 16. Application: For the complete job listing and to apply, click here.\xa0'

In [ ]:
# let's get the salary.
# let's get the location
# let's get whether it's a nonprofit or a government position


def get_title(job_description):
    return job_description.split('–')[0]

def get_salary(job_description):
    return job_description.split('Salary: ')[1]

get_title(jobs.loc[0, 'description'])
# get_salary(jobs.loc[0, 'description'])

# this isn't gonna work!

'Elections Systems Administrator, Arapahoe County, Colorado '

In [4]:
# Weather API

from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps

# ---------- FREE API KEY examples ---------------------

owm = OWM('c721ef57a321d8ae12fdaa16de2870d8')
mgr = owm.weather_manager()


# Search for current weather in London (Great Britain) and get details
observation = mgr.weather_at_place('Washington,DC,USA')
w = observation.weather

w.detailed_status         # 'clouds'
w.wind()                  # {'speed': 4.6, 'deg': 330}
w.humidity                # 87
w.temperature('celsius')  # {'temp_max': 10.5, 'temp': 9.7, 'temp_min': 9.0}
w.rain                    # {}
w.heat_index              # None
w.clouds                  # 75

w.temperature('fahrenheit')['temp']

44.91

In [6]:
import json
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure."},
        {"role": "user", "content": "..."}
    ],
    text={
        "format": {
              "type": "json_schema",
              "name": "research_paper_extraction",
              "schema": {
                "type": "object",
                "properties": {
                    "title": { "type": "string" },
                    "authors": { 
                        "type": "array",
                        "items": { "type": "string" }
                    },
                    "abstract": { "type": "string" },
                    "keywords": { 
                        "type": "array", 
                        "items": { "type": "string" }
                    }
                },
                "required": ["title", "authors", "abstract", "keywords"],
                "additionalProperties": False
            },
            "strict": True
        },
    },
)

research_paper = json.loads(response.output_text)
research_paper

{'title': 'A Comprehensive Study on Quantum Computing Paradigms',
 'authors': ['Dr. Jane Smith', 'Dr. Alan Turing', 'Prof. Maria Curie'],
 'abstract': 'This paper explores various paradigms of quantum computing, including quantum bits and entanglement. The study evaluates the potential impact on encryption, data processing, and computational efficiency. A comparative analysis with classical computing models highlights the unique advantages and limitations of quantum approaches.',
 'keywords': ['Quantum Computing',
  'Entanglement',
  'Quantum Bits',
  'Computational Efficiency',
  'Encryption']}

In [11]:
# sample description string
description = jobs.loc[1, 'description']
description

'Ballot Processing Supervisor, Boulder County, Colorado– The Boulder County Clerk and Recorder’s Office is currently seeking a Ballot Processing Supervisor to join our team. This vital role supports the execution of accurate and compliant elections within Boulder County. The Ballot Processing Supervisor is accountable for the execution of ballot processing, temporary staff scheduling and training, and public data requests for Boulder County. This position reports to the Ballot Processing Manager and works with members of the full-time elections staff as well as temporary election judges during an election. The position must comply with state laws and Secretary of State rules. This position is key to our ability to implement successful and trusted elections. The Ballot Processing Supervisor is accountable for driving bipartisanship oversight of election processes through implementing organized, efficient, and accurate elections with a well-trained temporary staff of judges and workers. 

In [12]:
# modify schema to include the fields we want to extract

import json
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given a job description and should convert it into the given structure."},
        {"role": "user", "content": description}
    ],
    text={
        "format": {
              "type": "json_schema",
              "name": "job_title_extraction",
              "schema": {
                "type": "object",
                "properties": {
                    "job_title": { "type": "string" },
                    "job_location": { "type": "string" },
                    "employer": { "type": "string" },
                    "salary_low_end": { "type": "string" },
                    "responsible_for_administering_elections": { "type": "boolean" },
                    "nonprofit_or_government": { "type": "string" }
                },
                "required": ["job_title", "employer"],
                "additionalProperties": False
            },
            "strict": False
        },
    },
)

job = json.loads(response.output_text)
print(job)

{'job_title': 'Ballot Processing Supervisor', 'job_location': 'Boulder County, Colorado', 'employer': 'Boulder County Clerk and Recorder’s Office', 'salary_low_end': '51408', 'responsible_for_administering_elections': True, 'nonprofit_or_government': 'government'}


In [13]:
# modify schema to include the fields we want to extract

import json
from openai import OpenAI


def parse_job_description(job_description: str):

    client = OpenAI()

    response = client.responses.create(
        model="gpt-4o-2024-08-06",
        input=[
            {"role": "system", "content": "You are an expert at structured data extraction. You will be given a job description and should convert it into the given structure."},
            {"role": "user", "content": description}
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "job_title_extraction",
                "schema": {
                    "type": "object",
                    "properties": {
                        "job_title": { "type": "string" },
                        "job_location": { "type": "string" },
                        "employer": { "type": "string" },
                        "salary_low_end": { "type": "string" },
                        "responsible_for_administering_elections": { "type": "boolean" },
                        "nonprofit_or_government": { "type": "string" }
                    },
                    "required": ["job_title", "job_location", "employer",
                                 "salary_low_end", "responsible_for_administering_elections",
                                 "nonprofit_or_government"],
                    "additionalProperties": False
                },
                "strict": True
            },
        },
    )

    return json.loads(response.output_text)
job = parse_job_description(description)

In [14]:
shortened = jobs.head(3)

# loop through df, parsing each job description in the 'description' column, and adding the parsed data to new columns

for i, row in shortened.iterrows():
    description = row['description']
    # print(i, description)
    
    parsed = parse_job_description(description)
    
    shortened.loc[i, 'job_title'] = parsed['job_title']
    shortened.loc[i, 'job_location'] = parsed['job_location']
    shortened.loc[i, 'employer'] = parsed['employer']
    shortened.loc[i, 'salary_low_end'] = parsed['salary_low_end']
    shortened.loc[i, 'responsible_for_administering_elections'] = parsed['responsible_for_administering_elections']
    shortened.loc[i, 'nonprofit_or_government'] = parsed['nonprofit_or_government']

    

In [60]:
shortened = jobs.head(3)

# loop through df, parsing each job description in the 'description' column, and adding the parsed data to new columns


for i, row in shortened.iterrows():
    description = row['description']
    # print(i, description)
    
    parsed = parse_job_description(description)

    for field in parsed:
        shortened.loc[i, field] = parsed[field]
    
    # shortened.loc[i, 'job_title'] = parsed.job_title
    # shortened.loc[i, 'job_location'] = parsed.job_location
    # shortened.loc[i, 'employer'] = parsed.employer
    # shortened.loc[i, 'salary_low_end'] = parsed.salary_low_end
    # shortened.loc[i, 'responsible_for_administering_elections'] = parsed.responsible_for_administering_elections
    # shortened.loc[i, 'nonprofit_or_government'] = parsed.nonprofit_or_government

shortened

,year,date,description,job_title,job_location,employer,salary_low_end,responsible_for_administering_elections,nonprofit_or_government
0,2025,04-03,"Elections Systems Administrator, Arapahoe County, Colorado – The Election Systems Administrator performs leadership level administrative, and professional work in carrying out a comprehensive public facing service operation. This position specifically leads and supports all areas of ballot proce...",Elections Systems Administrator,"Arapahoe County, Colorado",Arapahoe County,"$64,293.06",True,government
1,2025,03-27,"Ballot Processing Supervisor, Boulder County, Colorado– The Boulder County Clerk and Recorder’s Office is currently seeking a Ballot Processing Supervisor to join our team. This vital role supports the execution of accurate and compliant elections within Boulder County. The Ballot Processing Sup...",Ballot Processing Supervisor,"Boulder County, Colorado",Boulder County Clerk and Recorder’s Office,"$51,408",True,Government
2,2025,03-27,"Deputy Director, Minnesota Secretary of State–The Office of the Minnesota Secretary of State (OSS) is seeking a Deputy Elections Director that supports the Elections Director in providing division-wide strategic and administrative management, along with program and staff management, of all activ...",Deputy Elections Director,Minnesota,Office of the Minnesota Secretary of State,"$78,508 annually ($37.60 hourly)",True,Government
